In [ ]:
# Investment Research — Ingestion Notebook (No Caching)

# This notebook lets you:
# 1. Define tickers and a date range
# 2. Download **daily prices** (tidy format)
# 3. Download **full raw fundamentals** for each ticker
# 4. Save outputs under `data/raw/prices/` and `data/raw/fundamentals/`
# 5. Inspect results


In [1]:
# ---- Imports and Paths ----
import os
from datetime import datetime
import json
import pandas as pd
import yfinance as yf
from yahooquery import Ticker
import os
from pathlib import Path



# Method 1: Using pathlib.Path throughout (recommended)
NB_CWD = Path.cwd()
PROJECT_ROOT = NB_CWD.parent
DATA_DIR = NB_CWD.parent / 'data' / 'raw'
PRICES_DIR = DATA_DIR / "prices"
FUNDS_DIR = DATA_DIR / "fundamentals"

# Create directories using Path objects
for d in (DATA_DIR, PRICES_DIR, FUNDS_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Notebook CWD:", NB_CWD)
print("DATA_DIR:", DATA_DIR)
print("Project root:", PROJECT_ROOT)
print("PRICES_DIR:", PRICES_DIR)
print("FUNDS_DIR:", FUNDS_DIR)

Notebook CWD: /Users/danberle/Documents/projects/investment_analyzer/scripts
DATA_DIR: /Users/danberle/Documents/projects/investment_analyzer/data/raw
Project root: /Users/danberle/Documents/projects/investment_analyzer
PRICES_DIR: /Users/danberle/Documents/projects/investment_analyzer/data/raw/prices
FUNDS_DIR: /Users/danberle/Documents/projects/investment_analyzer/data/raw/fundamentals


In [2]:
# ---- Download Prices Function ----
def get_prices(tickers, start: str, end: str, interval: str = "1d", save: bool = True, auto_adjust: bool = False):
    """
    Download tidy daily OHLCV for a list of tickers by looping single-ticker requests.
    Returns one long/tidy DataFrame and (optionally) writes per-ticker CSVs.
    
    Args:
        tickers: List of ticker symbols or single ticker string
        start: Start date in YYYY-MM-DD format
        end: End date in YYYY-MM-DD format
        interval: Data interval (default "1d")
        save: Whether to save individual CSV files
        auto_adjust: Whether to auto-adjust prices
    
    Returns:
        pd.DataFrame: Combined tidy DataFrame with all ticker data
    """
    # Ensure tickers is a list
    if isinstance(tickers, str):
        tickers = [tickers]
    
    all_prices = []
    ordered_columns = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "ticker"]
    
    for tk in tickers:
        try:
            print(f"📈 Downloading {tk}...")
            
            # Download data for single ticker to avoid MultiIndex issues
            df = yf.download(
                tk, 
                start=start, 
                end=end, 
                interval=interval,
                auto_adjust=auto_adjust,
                progress=False  # Suppress progress bar for cleaner output
            )
            
            # Handle empty DataFrame
            if df.empty:
                print(f"⚠️  No data found for {tk}")
                continue
            
            # Reset index to make Date a column
            df = df.reset_index()
            
            # Handle potential MultiIndex columns (flatten if necessary)
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = [col[0] if col[1] == tk or col[1] == '' else f"{col[0]}_{col[1]}" 
                             for col in df.columns]
            
            # Ensure column names are clean
            df.columns = df.columns.str.strip()
            
            # Add ticker column
            df["ticker"] = tk
            
            # Standardize Date column name (yfinance sometimes returns 'Datetime')
            if 'Datetime' in df.columns:
                df = df.rename(columns={'Datetime': 'Date'})
            
            # Keep only columns that exist in our ordered list
            available_columns = [c for c in ordered_columns if c in df.columns]
            df = df[available_columns]
            
            # Ensure Date is datetime type
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'])
            
            # Sort by date
            if 'Date' in df.columns:
                df = df.sort_values('Date')
            
            # Save individual CSV if requested
            if save:
                clean_start = start.replace('-', '')
                clean_end = end.replace('-', '')
                filename = f"{tk.lower()}_{clean_start}_{clean_end}_{interval}.csv"
                filepath = PRICES_DIR / filename
                df.to_csv(filepath, index=False)
                print(f"✅ Saved {len(df)} records for {tk} → {filename}")
            
            all_prices.append(df)
            
        except Exception as e:
            print(f"❌ Error downloading {tk}: {str(e)}")
            continue
    
    # Combine all DataFrames
    if all_prices:
        combined = pd.concat(all_prices, ignore_index=True)
        # Sort by ticker and date for better organization
        if 'Date' in combined.columns and 'ticker' in combined.columns:
            combined = combined.sort_values(['ticker', 'Date']).reset_index(drop=True)
        
        print(f"🎉 Successfully combined data for {len(combined['ticker'].unique())} tickers, {len(combined)} total records")
        return combined
    else:
        print("⚠️  No data was successfully downloaded")
        return pd.DataFrame()

In [3]:

# ---- Download Prices Function (yahooquery - RECOMMENDED) ----
def get_prices(tickers, start: str, end: str, interval: str = "1d", save: bool = True):
    """
    Download tidy daily OHLCV for a list of tickers using yahooquery.
    Returns one long/tidy DataFrame and (optionally) writes per-ticker CSVs.
    
    Args:
        tickers: List of ticker symbols or single ticker string
        start: Start date in YYYY-MM-DD format
        end: End date in YYYY-MM-DD format
        interval: Data interval (default "1d")
        save: Whether to save individual CSV files
    
    Returns:
        pd.DataFrame: Combined tidy DataFrame with all ticker data
    """
    if isinstance(tickers, str):
        tickers = [tickers]
    
    all_prices = []
    ordered_columns = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "ticker"]
    
    for tk in tickers:
        try:
            print(f"📈 Downloading {tk}...")
            
            ticker_obj = Ticker(tk)
            df = ticker_obj.history(start=start, end=end, interval=interval)
            
            if df.empty:
                print(f"⚠️  No data found for {tk}")
                continue
            
            # Reset index to get date as column
            df = df.reset_index()
            
            # Add ticker column
            df["ticker"] = tk
            
            # Rename columns to match standard format
            column_mapping = {
                'date': 'Date',
                'open': 'Open', 
                'high': 'High',
                'low': 'Low',
                'close': 'Close',
                'adjclose': 'Adj Close',
                'volume': 'Volume'
            }
            df = df.rename(columns=column_mapping)
            
            # Keep only available columns
            available_columns = [c for c in ordered_columns if c in df.columns]
            df = df[available_columns]
            
            # Ensure Date is datetime
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'])
                df = df.sort_values('Date')
            
            if save:
                clean_start = start.replace('-', '')
                clean_end = end.replace('-', '')
                filename = f"{tk.lower()}_{clean_start}_{clean_end}_{interval}.csv"
                filepath = PRICES_DIR / filename
                df.to_csv(filepath, index=False)
                print(f"✅ Saved {len(df)} records for {tk} → {filename}")
            
            all_prices.append(df)
            
        except Exception as e:
            print(f"❌ Error downloading {tk}: {str(e)}")
            continue
    
    if all_prices:
        combined = pd.concat(all_prices, ignore_index=True)
        if 'Date' in combined.columns and 'ticker' in combined.columns:
            combined = combined.sort_values(['ticker', 'Date']).reset_index(drop=True)
        
        print(f"🎉 Successfully combined data for {len(combined['ticker'].unique())} tickers, {len(combined)} total records")
        return combined
    else:
        print("⚠️  No data was successfully downloaded")
        return pd.DataFrame()

# ---- Fallback using yfinance (if needed) ----
def get_prices_yfinance_fallback(tickers, start: str, end: str, interval: str = "1d", save: bool = True):
    """
    Fallback function using yfinance if yahooquery fails.
    """
    if isinstance(tickers, str):
        tickers = [tickers]
    
    all_prices = []
    ordered_columns = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "ticker"]
    
    for tk in tickers:
        try:
            print(f"📈 Downloading {tk} via yahooquery...")
            
            ticker_obj = Ticker(tk)
            df = ticker_obj.history(start=start, end=end, interval=interval)
            
            if df.empty:
                print(f"⚠️  No data found for {tk}")
                continue
            
            # Reset index to get date as column
            df = df.reset_index()
            
            # Add ticker column
            df["ticker"] = tk
            
            # Rename columns to match yfinance format
            column_mapping = {
                'date': 'Date',
                'open': 'Open', 
                'high': 'High',
                'low': 'Low',
                'close': 'Close',
                'adjclose': 'Adj Close',
                'volume': 'Volume'
            }
            df = df.rename(columns=column_mapping)
            
            # Keep only available columns
            available_columns = [c for c in ordered_columns if c in df.columns]
            df = df[available_columns]
            
            # Ensure Date is datetime
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'])
                df = df.sort_values('Date')
            
            if save:
                clean_start = start.replace('-', '')
                clean_end = end.replace('-', '')
                filename = f"{tk.lower()}_{clean_start}_{clean_end}_{interval}_yq.csv"
                filepath = PRICES_DIR / filename
                df.to_csv(filepath, index=False)
                print(f"✅ Saved {len(df)} records for {tk} → {filename}")
            
            all_prices.append(df)
            
        except Exception as e:
            print(f"❌ Error downloading {tk}: {str(e)}")
            continue
    
    if all_prices:
        combined = pd.concat(all_prices, ignore_index=True)
        if 'Date' in combined.columns and 'ticker' in combined.columns:
            combined = combined.sort_values(['ticker', 'Date']).reset_index(drop=True)
        
        print(f"🎉 Successfully combined data for {len(combined['ticker'].unique())} tickers, {len(combined)} total records")
        return combined
    else:
        return pd.DataFrame()

In [4]:
# ---- Helper Functions for Fundamentals ----
def _safe_get_block(block, ticker):
    """
    yahooquery properties (price, summary_detail, etc.) may be dict, DataFrame, or None.
    Return a dict-like (or basic Python structure) for JSON serialization.
    """
    # Most of these properties are dicts keyed by ticker
    if isinstance(block, dict):
        return block.get(ticker)
    # Some versions may return a pandas object or list-like; convert if needed
    if hasattr(block, "to_dict"):
        try:
            return block.to_dict()
        except Exception:
            pass
    return block  # could be None or plain type

def _safe_stmt(callable_fn):
    """
    Statement methods (income_statement/balance_sheet/cash_flow) vary by version.
    Try a few signatures and return a dict keyed by ticker OR a serializable structure.
    """
    for args in ((), ()):
        for kwargs in ({}, {"trailing": False}):
            try:
                return callable_fn(*args, **kwargs)
            except TypeError:
                continue
            except Exception:
                return None
    return None

def _flatten_fundamentals_dict(ticker: str, raw_dict: dict) -> pd.DataFrame:
    """Light flatten for a quick one-row preview (deep cleaning happens later)."""
    rows = {"ticker": ticker, "as_of": raw_dict.get("as_of")}
    for block in ["price", "summary_detail", "key_stats", "financial_data", "asset_profile"]:
        b = raw_dict.get(block)
        if isinstance(b, dict):
            pref = f"{block}."
            rows.update({pref + k: v for k, v in b.items()})
    return pd.DataFrame([rows])

In [5]:
# ---- Fundamentals Functions ----
def download_fundamentals_raw(ticker: str, save_json: bool = True, save_flat_csv: bool = True):
    t = Ticker(ticker)

    # property blocks
    price            = _safe_get_block(getattr(t, "price", None), ticker) if hasattr(t, "price") else None
    summary_detail   = _safe_get_block(getattr(t, "summary_detail", None), ticker) if hasattr(t, "summary_detail") else None
    key_stats        = _safe_get_block(getattr(t, "key_stats", None), ticker) if hasattr(t, "key_stats") else None
    financial_data   = _safe_get_block(getattr(t, "financial_data", None), ticker) if hasattr(t, "financial_data") else None
    asset_profile    = _safe_get_block(getattr(t, "asset_profile", None), ticker) if hasattr(t, "asset_profile") else None
    earnings         = _safe_get_block(getattr(t, "earnings", None), ticker) if hasattr(t, "earnings") else None

    # statement blocks (may be dict keyed by ticker or something else)
    income_stmt_raw  = _safe_stmt(getattr(t, "income_statement", lambda: None)) if hasattr(t, "income_statement") else None
    balance_raw      = _safe_stmt(getattr(t, "balance_sheet", lambda: None))    if hasattr(t, "balance_sheet")    else None
    cashflow_raw     = _safe_stmt(getattr(t, "cash_flow", lambda: None))        if hasattr(t, "cash_flow")        else None

    # Extract this ticker's piece if we got dicts keyed by ticker
    if isinstance(income_stmt_raw, dict):
        income_stmt_raw = income_stmt_raw.get(ticker)
    if isinstance(balance_raw, dict):
        balance_raw = balance_raw.get(ticker)
    if isinstance(cashflow_raw, dict):
        cashflow_raw = cashflow_raw.get(ticker)

    # If any of those are pandas objects, convert to serializable records
    def to_serializable(obj):
        if hasattr(obj, "to_dict"):
            try:
                return obj.to_dict(orient="records")
            except TypeError:
                return obj.to_dict()
        return obj

    raw = {
        "as_of": datetime.today().strftime("%Y-%m-%d"),
        "price": price,
        "summary_detail": summary_detail,
        "key_stats": key_stats,
        "financial_data": financial_data,
        "asset_profile": asset_profile,
        "earnings": earnings,
        "income_statement": to_serializable(income_stmt_raw),
        "balance_sheet": to_serializable(balance_raw),
        "cash_flow": to_serializable(cashflow_raw),
    }

    if save_json:
        asof = datetime.today().strftime("%Y%m%d")
        raw_fp = FUNDS_DIR / f"{ticker.lower()}_fundamentals_raw_{asof}.json"
        with raw_fp.open("w") as f:
            json.dump(raw, f, indent=2, default=str)
        print(f"✅ Saved RAW fundamentals for {ticker} → {raw_fp}")

    flat = _flatten_fundamentals_dict(ticker, raw)
    if save_flat_csv:
        snap_fp = FUNDS_DIR / f"{ticker.lower()}_fundamentals_flat_{datetime.today().strftime('%Y%m%d')}.csv"
        flat.to_csv(snap_fp, index=False)
        print(f"📄 Saved flattened snapshot for {ticker} → {snap_fp}")

    return flat

In [6]:
# ---- Parameters ----
# Edit as you like
tickers = ["AAPL", "MSFT", "GOOGL"]
start_date = "2020-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")
interval = "1d"

tickers, start_date, end_date, interval

(['AAPL', 'MSFT', 'GOOGL'], '2020-01-01', '2025-08-12', '1d')

In [8]:
# 1) Prices loop (independent; won't block fundamentals if one fails)
price_results = []
for tk in tickers:
    try:
        dfp = get_prices([tk], start_date, end_date, interval, save=True)
        price_results.append(dfp)
    except Exception as e:
        print(f"❌ Price fetch failed for {tk}: {e}")

prices_df = pd.concat(price_results, ignore_index=True) if price_results else pd.DataFrame()

prices_df.shape


📈 Downloading AAPL...
✅ Saved 1409 records for AAPL → aapl_20200101_20250812_1d.csv
🎉 Successfully combined data for 1 tickers, 1409 total records
📈 Downloading MSFT...
✅ Saved 1409 records for MSFT → msft_20200101_20250812_1d.csv
🎉 Successfully combined data for 1 tickers, 1409 total records
📈 Downloading GOOGL...
✅ Saved 1409 records for GOOGL → googl_20200101_20250812_1d.csv
🎉 Successfully combined data for 1 tickers, 1409 total records


(4227, 8)

In [11]:
# Save combined CSV with simplified naming
if not prices_df.empty:
    clean_start = start_date.replace('-', '')
    clean_end = end_date.replace('-', '')
    
    combined_filename = f"combined_prices_{clean_start}_{clean_end}.csv"
    combined_filepath = PRICES_DIR / combined_filename
    
    prices_df.to_csv(combined_filepath, index=False)
    print(f"✅ Saved combined data: {combined_filename}")
    print(f"📊 Total records: {len(prices_df)}")
    print(f"📈 Tickers: {', '.join(prices_df['ticker'].unique())}")
else:
    print("⚠️ No data to save - combined DataFrame is empty")

✅ Saved combined data: combined_prices_20200101_20250812.csv
📊 Total records: 4227
📈 Tickers: AAPL, MSFT, GOOGL


In [9]:
# 2) Fundamentals loop (independent)
fund_results = []
for tk in tickers:
    try:
        dff = download_fundamentals_raw(tk, save_json=True, save_flat_csv=True)
        fund_results.append(dff)
    except Exception as e:
        print(f"❌ Fundamentals fetch failed for {tk}: {e}")

funds_df = pd.concat(fund_results, ignore_index=True) if fund_results else pd.DataFrame()

funds_df.shape

✅ Saved RAW fundamentals for AAPL → /Users/danberle/Documents/projects/investment_analyzer/data/raw/fundamentals/aapl_fundamentals_raw_20250812.json
📄 Saved flattened snapshot for AAPL → /Users/danberle/Documents/projects/investment_analyzer/data/raw/fundamentals/aapl_fundamentals_flat_20250812.csv
✅ Saved RAW fundamentals for MSFT → /Users/danberle/Documents/projects/investment_analyzer/data/raw/fundamentals/msft_fundamentals_raw_20250812.json
📄 Saved flattened snapshot for MSFT → /Users/danberle/Documents/projects/investment_analyzer/data/raw/fundamentals/msft_fundamentals_flat_20250812.csv
✅ Saved RAW fundamentals for GOOGL → /Users/danberle/Documents/projects/investment_analyzer/data/raw/fundamentals/googl_fundamentals_raw_20250812.json
📄 Saved flattened snapshot for GOOGL → /Users/danberle/Documents/projects/investment_analyzer/data/raw/fundamentals/googl_fundamentals_flat_20250812.csv


(3, 178)

In [13]:
# Save combined CSV with simplified naming
if not funds_df.empty:
    clean_date = end_date.replace('-', '')
    
    combined_filename = f"combined_fundamentals_{clean_date}.csv"
    combined_filepath = FUNDS_DIR / combined_filename
    
    funds_df.to_csv(combined_filepath, index=False)
    print(f"✅ Saved combined data: {combined_filename}")
    print(f"📊 Total records: {len(funds_df)}")
    print(f"📈 Tickers: {', '.join(funds_df['ticker'].unique())}")
else:
    print("⚠️ No data to save - combined DataFrame is empty")

✅ Saved combined data: combined_fundamentals_20250812.csv
📊 Total records: 3
📈 Tickers: AAPL, MSFT, GOOGL


In [10]:
# Quick Preview
display(prices_df.head(8))
display(funds_df.head(3))

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,72.538521,135480400,AAPL
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,71.833298,146322800,AAPL
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,72.405678,118387200,AAPL
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,72.065147,108872000,AAPL
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,73.224403,132079200,AAPL
5,2020-01-09,76.809998,77.607498,76.550003,77.407501,74.779732,170108400,AAPL
6,2020-01-10,77.650002,78.167503,77.062500,77.582497,74.948776,140644800,AAPL
7,2020-01-13,77.910004,79.267502,77.787498,79.239998,76.550041,121532000,AAPL


,ticker,as_of,price.maxAge,price.preMarketChangePercent,price.preMarketChange,price.preMarketTime,price.preMarketPrice,price.preMarketSource,price.postMarketChangePercent,price.postMarketChange,...,asset_profile.auditRisk,asset_profile.boardRisk,asset_profile.compensationRisk,asset_profile.shareHolderRightsRisk,asset_profile.overallRisk,asset_profile.governanceEpochDate,asset_profile.compensationAsOfEpochDate,asset_profile.irWebsite,asset_profile.executiveTeam,asset_profile.maxAge
0,AAPL,2025-08-12,1,-0.005326,-1.209992,2025-08-12 05:29:40,225.97,FREE_REALTIME,-0.004054,-0.921097,...,7,1,3,1,1,2025-07-31 20:00:00,2024-12-30 19:00:00,http://investor.apple.com/,[],86400
1,MSFT,2025-08-12,1,0.002884,1.505005,2025-08-12 05:29:17,523.42,FREE_REALTIME,0.001102,0.574951,...,9,5,4,2,3,2025-07-31 20:00:00,2024-12-30 19:00:00,http://www.microsoft.com/investor/default.aspx,[],86400
2,GOOGL,2025-08-12,1,-0.000647,-0.130005,2025-08-12 05:29:10,200.87,FREE_REALTIME,-0.001741,-0.350006,...,7,9,10,10,10,2025-07-31 20:00:00,2024-12-30 19:00:00,NaN,[],86400
